In [5]:
import torch
import torch.nn as nn
from torch_geometric.nn import GCNConv, Linear, SAGEConv, GATv2Conv, GATConv
from torch_geometric.data import Data, Dataset
from torch_geometric.loader import DataLoader
import torch.nn.functional as F

import networkx as nx
import numpy as np
from pathlib import Path
import scipy as sp
rng = np.random.default_rng()

/Users/sbharg/homework/ut_austin/ece381k_mlnetworks/gnn_influence_probs/.venv/lib/python3.12/site-packages/torch_geometric/typing.py:68: UserWarning: An issue occurred while importing 'pyg-lib'. Disabling its usage. Stacktrace: dlopen(/Users/sbharg/homework/ut_austin/ece381k_mlnetworks/gnn_influence_probs/.venv/lib/python3.12/site-packages/libpyg.so, 0x0006): Library not loaded: /Library/Frameworks/Python.framework/Versions/3.12/Python
  Referenced from: <E87A820F-D734-3F45-AFBE-9D80043A97C0> /Users/sbharg/homework/ut_austin/ece381k_mlnetworks/gnn_influence_probs/.venv/lib/python3.12/site-packages/libpyg.so
  Reason: tried: '/Library/Frameworks/Python.framework/Versions/3.12/Python' (no such file), '/System/Volumes/Preboot/Cryptexes/OS/Library/Frameworks/Python.framework/Versions/3.12/Python' (no such file), '/Library/Frameworks/Python.framework/Versions/3.12/Python' (no such file)
  warnings.warn(f"An issue occurred while importing 'pyg-lib'. "


In [6]:
def compute_cascade_likelihood(edge_probs, edge_index, cascade, epsilon=1e-8):
  """
  Compute the likelihood of observing a single cascade given edge probabilities.
  
  Args:
  - edge_probs: Tensor of predicted edge probabilities
  - edge_index: Tensor of shape [2, num_edges] containing edge indices
  - cascade: List of lists, where each inner list contains nodes activated at that time step
  - epsilon: Small value to avoid log(0)
  
  Returns:
  - log_likelihood: Log-likelihood of the cascade
  """
  device = edge_probs.device
  num_nodes = edge_index.max().item() + 1
  activated = torch.zeros(num_nodes, dtype=torch.bool, device=device)
  log_likelihood = 0.0

  src, dst = edge_index

  for t in range(len(cascade)):
    prev_activated = torch.tensor(cascade[t-1] if t-1 >= 0 else [], device=device)
    curr_activated = torch.tensor(cascade[t], device=device)
    next_activated = torch.tensor(cascade[t+1] if t+1 < len(cascade) else [], device=device)
    activated[curr_activated] = True
    
    # Probability of activation from parents
    for v in curr_activated:
      parents = src[(dst == v) & activated[src]]
      activated_parents = parents[torch.isin(parents, prev_activated)]
      if len(activated_parents) > 0:
        prob_v_activated = 1 - torch.prod(1 - edge_probs[torch.isin(src, activated_parents) & (dst == v)])
        log_likelihood += torch.log(prob_v_activated + epsilon)

    
    # Probability of non-activation of children
    for v in curr_activated:
      children = dst[(src == v) & ~activated[dst]]
      non_activated_children = children[~torch.isin(children, next_activated)]
      if len(non_activated_children) > 0:
        prob_children_not_activated = torch.prod(1 - edge_probs[(src == v) & torch.isin(dst, non_activated_children)])
        log_likelihood += torch.log(prob_children_not_activated + epsilon)
    

  return log_likelihood

  '''
  device = edge_probs.device
  num_nodes = edge_index.max().item() + 1
  activated = torch.zeros(num_nodes, dtype=torch.bool, device=device)
  log_likelihood = 0.0

  for t, activated_nodes in enumerate(cascade):
    if t == 0:
      activated[activated_nodes] = True
      continue

    # Compute activation probabilities for this step
    src, dst = edge_index
    mask = activated[src] & ~activated[dst]
    relevant_probs = edge_probs[mask]
    relevant_dst = dst[mask]

    # Compute likelihood of activations and non-activations
    new_activations = torch.tensor(activated_nodes, device=device)
    activated_probs = relevant_probs[torch.isin(relevant_dst, new_activations)]
    non_activated_probs = relevant_probs[~torch.isin(relevant_dst, new_activations)]

    log_likelihood += torch.sum(torch.log(activated_probs + epsilon))
    log_likelihood += torch.sum(torch.log(1 - non_activated_probs + epsilon))

    # Update activated nodes
    activated[activated_nodes] = True

  return log_likelihood
  '''

def compute_loss(edge_probs, edge_index, cascades):
  """
  Compute the negative log-likelihood loss for multiple cascades.
  
  Args:
  - edge_probs: Tensor of predicted edge probabilities
  - edge_index: Tensor of shape [2, num_edges] containing edge indices
  - cascades: List of cascades, where each cascade is a list of lists of activated nodes
  
  Returns:
  - loss: Negative log-likelihood loss
  """
  total_log_likelihood = 0.0
  for cascade in cascades:
    total_log_likelihood += compute_cascade_likelihood(edge_probs, edge_index, cascade)
  
  # Return negative log-likelihood as the loss
  return -total_log_likelihood

In [7]:
def create_dataset(G: nx.DiGraph, features):
  # Create a PyG Data object from the networkx graph
  edge_index = torch.tensor(list(G.edges)).t().contiguous()
  #x = torch.tensor(features, dtype=torch.float)
  data = Data(x=features, edge_index=edge_index)
  return data

n = 100
p = 0.1
gname = f"er_{n}_{str(p).replace('.', '')}"
path = Path(f"datasets/synthetic/{gname}")

with open(path / f"{gname}.mtx", "rb") as fh:
  G = nx.from_scipy_sparse_array(sp.io.mmread(fh), create_using=nx.DiGraph)
with open(path / "feats.npy", "rb") as fh:
  features_npy = torch.tensor(np.load(fh), dtype=torch.float)

cascades = []
idxes = rng.choice(500, 100, replace=False)
for i in idxes:
  with open(path / f"diffusions/timestamps/{i}.txt", "r") as fh:
    cascade = []
    for line in fh:
      cascade.append(list(map(int, line.strip().split())))
    cascades.append(cascade)

In [8]:
class GNNIndependentCascade(torch.nn.Module):
  def __init__(self, num_node_features, hidden_dim, num_layers=2):
    super(GNNIndependentCascade, self).__init__()
    self.num_layers = num_layers
    self.convs = nn.ModuleList([
      GATConv(num_node_features if i == 0 else hidden_dim, hidden_dim) 
      for i in range(num_layers)]
    )

    self.edge_predictor = nn.Sequential(
      nn.Linear(2 * hidden_dim, hidden_dim),
      nn.ELU(),
      nn.Linear(hidden_dim, 1),
      nn.ELU()
    )

  def forward(self, data):
    x, edge_index = data.x, data.edge_index

    # Node embedding
    for i in range(self.num_layers):
      x = self.convs[i](x, edge_index)
      x = torch.relu(x)
      x = torch.dropout(x, p=0.1, train=self.training)

    # Edge probability prediction
    row, col = edge_index
    edge_features = torch.cat([x[row], x[col]], dim=1)
    edge_probs = torch.sigmoid(self.edge_predictor(edge_features).squeeze())
    #edge_probs = torch.sigmoid(torch.sum(x[row] * x[col], dim=1))

    return edge_probs

def train_model(model, optimizer, data, cascades, num_epochs, batch_size = 50):
  model.train()
  #batches = DataLoader(cascades, batch_size=10, shuffle=True)
  #print(batches)

  for epoch in range(num_epochs):
    loss = 0.0
    rng.shuffle(cascades)
    batches = [cascades[i:i+batch_size] for i in range(0, len(cascades), batch_size)]

    for batch in batches:
      optimizer.zero_grad()
      edge_probs = model.forward(data)

      batch_loss = compute_loss(edge_probs, data.edge_index, batch)
      batch_loss.backward()
      optimizer.step()
      loss += batch_loss.item()
    
    if epoch % 10 == 0 or epoch == num_epochs - 1:
      print(f"Epoch {epoch+1}/{num_epochs}, Total Loss: {loss:.4f}")
      #print(edge_probs[0])
      #print(edge_probs[1])
      #print('\n')

features_eye = torch.eye(G.number_of_nodes())
data = create_dataset(G, features_eye)
model = GNNIndependentCascade(data.num_features, 64)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=5e-4)
train_model(model, optimizer, data, cascades, 100)

Epoch 1/100, Total Loss: 12887.4165
Epoch 11/100, Total Loss: 11622.9136


KeyboardInterrupt: 

In [62]:
model.eval()
l1_error = 0
l2_error = 0
edge_probs = model(data)

for i, e in enumerate(G.edges()):
  u, v = e
  p = G[u][v]['weight']
  l1_error += abs(p - edge_probs[i].item())
  l2_error += (p - edge_probs[i].item())**2

print(l1_error)
print(l1_error / G.number_of_edges())
print(edge_probs)

189.2186086857233
0.1960814597779516
tensor([0.3197, 0.3217, 0.3149, 0.3180, 0.3156, 0.3187, 0.3264, 0.3200, 0.3255,
        0.3219, 0.3153, 0.3224, 0.3187, 0.3131, 0.3198, 0.3184, 0.3220, 0.3142,
        0.3135, 0.3194, 0.3116, 0.3134, 0.3168, 0.3149, 0.3117, 0.3106, 0.3160,
        0.3094, 0.3082, 0.3115, 0.3128, 0.3181, 0.3187, 0.3106, 0.3135, 0.3131,
        0.3104, 0.3107, 0.3077, 0.3138, 0.3118, 0.3198, 0.3139, 0.3127, 0.3224,
        0.3148, 0.3216, 0.3104, 0.3053, 0.3076, 0.3064, 0.3042, 0.3045, 0.3060,
        0.3180, 0.3169, 0.3227, 0.3205, 0.3175, 0.3218, 0.3199, 0.3278, 0.3253,
        0.3255, 0.3250, 0.3224, 0.3169, 0.3216, 0.3146, 0.3051, 0.3126, 0.3175,
        0.3112, 0.3131, 0.3143, 0.3129, 0.3146, 0.3123, 0.3122, 0.3117, 0.3111,
        0.3154, 0.3197, 0.3196, 0.3158, 0.3187, 0.3144, 0.3142, 0.3152, 0.3120,
        0.3124, 0.3112, 0.3144, 0.3118, 0.3157, 0.3212, 0.3146, 0.3170, 0.3262,
        0.3225, 0.3274, 0.3195, 0.3168, 0.3189, 0.3163, 0.3143, 0.3166, 0.3203,
   

In [6]:
class GNNIndependentCascade(torch.nn.Module):
  def __init__(self, in_dim, hidden_dim, n_nodes, n_edges, num_layers=2):
    super(GNNIndependentCascade, self).__init__()
    self.num_layers = num_layers

    self.node_embed = nn.Embedding(n_nodes, in_dim)
    self.edge_embed = nn.Parameter(torch.rand((n_edges, hidden_dim)))

    self.convs = nn.ModuleList([
      SAGEConv(in_dim if i == 0 else hidden_dim, hidden_dim) 
      for i in range(num_layers)
    ])

    self.edge_predictor = nn.Sequential(
      nn.Linear(hidden_dim, hidden_dim),
      nn.ELU(),
      nn.Linear(hidden_dim, 1),
    )
    #nn.Linear(hidden_dim, 1)

  def forward(self, data):
    x, edge_index = data.x, data.edge_index

    # Node embedding
    #for i in range(self.num_layers):
    #  x = self.convs[i](x, edge_index)
    #  x = torch.relu(x)
    #  x = torch.dropout(x, p=0.1, train=self.training)

    # Edge probability prediction
    edge_probs = torch.sigmoid(self.edge_predictor(self.edge_embed))

    #edge_probs = torch.sigmoid(torch.sum(x[row] * x[col], dim=1))

    return edge_probs

def train_model(model, optimizer, data, cascades, num_epochs, batch_size = 50):
  model.train()
  #batches = DataLoader(cascades, batch_size=10, shuffle=True)
  #print(batches)

  for epoch in range(num_epochs):
    loss = 0.0
    rng.shuffle(cascades)
    batches = [cascades[i:i+batch_size] for i in range(0, len(cascades), batch_size)]

    for batch in batches:
      optimizer.zero_grad()
      edge_probs = model.forward(data)

      batch_loss = compute_loss(edge_probs, data.edge_index, batch)
      batch_loss.backward()
      optimizer.step()
      loss += batch_loss.item()
    
    if epoch % 10 == 0 or epoch == num_epochs - 1:
      print(f"Epoch {epoch+1}/{num_epochs}, Total Loss: {loss:.4f}")
      #print(edge_probs[0])
      #print(edge_probs[1])
      #print('\n')

n = G.number_of_nodes()
features_embed = nn.Embedding(n, n)
data = create_dataset(G, features_embed(torch.arange(n)))
model = GNNIndependentCascade(data.num_features, 64, len(data.x), len(data.edge_index[0]), num_layers=2)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)
train_model(model, optimizer, data, cascades, 100)

Epoch 1/100, Total Loss: 13271.6597
Epoch 11/100, Total Loss: 10534.9819
Epoch 21/100, Total Loss: 9552.8940
Epoch 31/100, Total Loss: 9453.4146
Epoch 41/100, Total Loss: 9430.0796
Epoch 51/100, Total Loss: 9422.7651
Epoch 61/100, Total Loss: 9424.0000
Epoch 71/100, Total Loss: 9418.0762
Epoch 81/100, Total Loss: 9417.8579
Epoch 91/100, Total Loss: 9419.9067
Epoch 100/100, Total Loss: 9419.1880


In [7]:
model.eval()
l1_error = 0
l2_error = 0
edge_probs = model(data)

for i, e in enumerate(G.edges()):
  u, v = e
  p = G[u][v]['weight']
  l1_error += abs(p - edge_probs[i].item())
  l2_error += (p - edge_probs[i].item())**2

print(l1_error)
print(l1_error / G.number_of_edges())
print(edge_probs)

95.2113658612902
0.0986646278355339
tensor([[6.1738e-02],
        [7.9006e-01],
        [4.0442e-01],
        [4.4102e-01],
        [8.2723e-01],
        [6.6342e-01],
        [3.3259e-01],
        [4.0768e-01],
        [1.7554e-01],
        [5.3576e-01],
        [5.7342e-01],
        [2.8838e-01],
        [2.6197e-01],
        [9.4849e-02],
        [2.8793e-01],
        [2.8896e-02],
        [2.3368e-01],
        [3.3511e-01],
        [1.2920e-03],
        [4.6355e-01],
        [2.6946e-01],
        [3.1625e-01],
        [1.6136e-01],
        [8.2319e-02],
        [7.2024e-01],
        [1.6765e-01],
        [1.4062e-01],
        [2.5346e-01],
        [1.4348e-01],
        [1.7394e-01],
        [5.1224e-01],
        [5.3003e-01],
        [7.0310e-02],
        [6.7588e-01],
        [4.4773e-01],
        [4.0013e-01],
        [6.8070e-02],
        [2.6520e-01],
        [1.3172e-03],
        [4.7475e-01],
        [3.6202e-01],
        [4.5034e-01],
        [5.1787e-01],
        [3.3955e-0

In [91]:
def create_dataset(G: nx.DiGraph, features):
  # Create a PyG Data object from the networkx graph
  edge_index = torch.tensor(list(G.edges)).t().contiguous()
  #x = torch.tensor(features, dtype=torch.float)
  data = Data(x=features, edge_index=edge_index)
  return data

n = 100
m0 = 2
p = 0.1
a = 0.41
b = 0.54
c = 0.05
gname = f"er_{n}_{str(p).replace('.', '')}"
#gname = "congress-twitter"
#gname = f"sf_{n}_{str(a).replace('.', '')}_{str(b).replace('.', '')}_{str(c).replace('.', '')}"
#gname = f"ba_{n}_{m0}"
path = Path(f"datasets/synthetic/{gname}")

with open(path / f"{gname}.mtx", "rb") as fh:
  G = nx.from_scipy_sparse_array(sp.io.mmread(fh), create_using=nx.DiGraph)

cascades = []
idxes = rng.choice(1000, 100, replace=False)
for i in idxes:
  with open(path / f"diffusions/timestamps/{i}.txt", "r") as fh:
    cascade = []
    for line in fh:
      cascade.append(list(map(int, line.strip().split())))
    cascades.append(cascade)

In [9]:
class GNNIndependentCascade(torch.nn.Module):
  def __init__(self, node_emb_dim, hidden_dim, n_nodes, n_edges, num_layers=2):
    super(GNNIndependentCascade, self).__init__()
    self.n = n_nodes
    self.m = n_edges

    self.num_layers = num_layers

    self.node_embed = nn.Embedding(self.n, node_emb_dim)
    self.edge_embed = nn.Parameter(torch.Tensor(self.n, self.n, hidden_dim))

    self.convs = nn.ModuleList([
      GATConv(node_emb_dim if i == 0 else hidden_dim, hidden_dim) 
      for i in range(num_layers)
    ])

    self.edge_predictor = nn.Sequential(
      nn.Linear(2*hidden_dim, hidden_dim),
      nn.ReLU(),
      nn.Linear(hidden_dim, 1),
    )
    #nn.Linear(hidden_dim, 1)

  def forward(self, data):
    edge_index = data.edge_index
    x = self.node_embed(torch.arange(self.n))

    src, dst = edge_index
    edge_emb = self.edge_embed[src, dst]

    # Node embedding
    for i in range(self.num_layers):
      #x = self.convs[i](x, edge_index, edge_emb)
      x = self.convs[i](x, edge_index)
      x = F.gelu(x)
      x = torch.dropout(x, p=0.1, train=self.training)

    # Edge probability prediction
    #edge_repr = torch.cat([x[src], x[dst]], dim=1)
    edge_repr = torch.cat([torch.add(x[src], x[dst]), edge_emb], dim=1)
    edge_probs = torch.sigmoid(self.edge_predictor(edge_repr))

    #edge_probs = torch.sigmoid(torch.sum(x[row] * x[col], dim=1))

    return edge_probs

def train_model(model, optimizer, data, cascades, num_epochs, batch_size = 50):
  model.train()
  #batches = DataLoader(cascades, batch_size=10, shuffle=True)
  #print(batches)

  for epoch in range(num_epochs):
    loss = 0.0
    rng.shuffle(cascades)
    batches = [cascades[i:i+batch_size] for i in range(0, len(cascades), batch_size)]

    for batch in batches:
      optimizer.zero_grad()
      edge_probs = model.forward(data)

      batch_loss = compute_loss(edge_probs, data.edge_index, batch)
      batch_loss.backward()
      optimizer.step()
      loss += batch_loss.item()
    
    if epoch % 10 == 0 or epoch == num_epochs - 1:
      print(f"Epoch {epoch+1}/{num_epochs}, Avg Loss: {loss / len(cascades):.4f}")

n = G.number_of_nodes()
m = G.number_of_edges()
features = torch.eye(n)
data = create_dataset(G, features)

model = GNNIndependentCascade(64, 16, n, m, num_layers=2)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)
train_model(model, optimizer, data, cascades, 50, len(cascades) // 2)

Epoch 1/50, Avg Loss: 121.3335
Epoch 11/50, Avg Loss: 103.9308
Epoch 21/50, Avg Loss: 102.3731


KeyboardInterrupt: 

In [137]:
model.eval()
edge_probs = model(data)

residuals = []
for i, e in enumerate(G.edges()):
  u, v = e
  p = G[u][v]['weight']
  residuals.append(abs(p - edge_probs[i].item()))

l1_error = sum(residuals) / len(residuals)
median = np.median(residuals)
quartile1 = np.percentile(residuals, 25) 
quartile3 = np.percentile(residuals, 75)

print(f"MAE Error: {l1_error}")
print(f"Median: {median}, Q1: {quartile1}, Q3: {quartile3}")
print(edge_probs)

MAE Error: 0.10014827800129067
Median: 0.08214325020790098, Q1: 0.03609595981955528, Q3: 0.1434874703884125
tensor([[0.2388],
        [0.1588],
        [0.5576],
        [0.1086],
        [0.4183],
        [0.6892],
        [0.1740],
        [0.3602],
        [0.0147],
        [0.1760],
        [0.7309],
        [0.2587],
        [0.3410],
        [0.1270],
        [0.4631],
        [0.3812],
        [0.3395],
        [0.5676],
        [0.2961],
        [0.5747],
        [0.1874],
        [0.4196],
        [0.4219],
        [0.3334],
        [0.2243],
        [0.1855],
        [0.3464],
        [0.4313],
        [0.3453],
        [0.5353],
        [0.2628],
        [0.7263],
        [0.1158],
        [0.2674],
        [0.0904],
        [0.4271],
        [0.0665],
        [0.2159],
        [0.4200],
        [0.5233],
        [0.6858],
        [0.0356],
        [0.0101],
        [0.1339],
        [0.7061],
        [0.3281],
        [0.3773],
        [0.4297],
        [0.0204],
        [0